In [1]:
%pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

In [3]:
from os import path
from sys import path as sys_path
parent_dir = path.abspath(path.join('..'))
if parent_dir not in sys_path:
    sys_path.append(parent_dir)
    print(f"Added {parent_dir.split("\\")[-1]} to sys.path")
from models.commit import Commit
from models.cf import CommitFile
from models.commit import Commit
from models.file import File
from typing import List

Added finer to sys.path


In [ ]:
def create_cfs(files: List[str], com: Commit, parent_dir: str):
    repo_path = path.join(parent_dir, com.org_name, com.repo_name)
    processed_cfs = set()
    
    futures = set()
    
    with ThreadPoolExecutor(max_workers=25) as executor:
        for file in files:
            cf_key = (file, com.sha, com.repo_name, com.org_name)
            if cf_key in processed_cfs:
                continue
            processed_cfs.add(cf_key)
            futures.add(executor.submit(File.get_file_content, repo_path, com.sha, file))
        
        if CommitFile.exists_in_batches(processed_cfs):
            return
        
        db_cfs = []
        for future in as_completed(futures):
            file_content, file_name = future.result()
            file_to_db = CommitFile(com.repo_name, com.org_name, file_name, com.sha, file_content)
            db_cfs.append(file_to_db)
        
        if db_cfs:
            CommitFile.add_cfs_in_batches(db_cfs)


In [5]:
commits = Commit.fetch_all_commits()
parent_folder = path.join('..', 'download', 'orgs')

In [ ]:
for com in tqdm(commits, total=len(commits), desc="Processing commits"):
    files = com.get_file_names_from_git(path.join(parent_folder, com.org_name, com.repo_name), com.sha)
    if files:
        create_cfs(files, com, parent_folder)

Processing commits:   8%|▊         | 2449/32533 [06:18<57:36,  8.70it/s]   